In [ ]:
import demand_supply_files_processing as dsfp
import pandas as pd
import numpy as np
import time


'''Importing all the files from the modules previously created to work with the data and the metrics and generating a solution'''

# st = time.time()
cluster_importance = dsfp.cluster_importance
weekly_cluster_demand = dsfp.weekly_demand_of_cluster
cluster_truck_capacity = dsfp.cluster_truck_capacity
stable_route_info, dynamic_route_info = dsfp.stable_route_info, dsfp.dynamic_route_info
demand_info = dsfp.demand_info
supply_info = dsfp.supply_info
truck_capacity = dsfp.truck_capacity
loading_capacity_stable  = dsfp.loading_capacity_stable
loading_capacity_dynamic = dsfp.loading_capacity_dynamic
trucks = dsfp.Trucks
packaging_type = dsfp.packaging_type
total_suppliers = dsfp.total_suppliers
total_clusters = dsfp.total_clusters
weeks = dsfp.Weeks
plants = dsfp.plants
suppliers = dsfp.supplier_list
arcs =  dsfp.Arcs
matrix = dsfp.matrix
supplier_sublist = dsfp.supplier_sublist
errors_clusters = dsfp.get_sum_of_error_deviations(weekly_cluster_demand,cluster_truck_capacity,total_clusters,weeks)
p2s,s2s=dsfp.p2s_distance,dsfp.s2s_distance

In [ ]:
'''creating different lists to store dynamic and stable route importance scores. this is just for data understanding and has not been used in the
heuristics'''


dynamic_route_clusters = np.array([a for a in dynamic_route_info.keys() if stable_route_info[a] == 0])
stable_route_clusters = np.array([a for a in stable_route_info.keys() if stable_route_info[a] != 0])

stable_route_importance_scores = {a:b for a,b in cluster_importance.items() if a in stable_route_clusters}
stable_route_importance_scores = dict(sorted(stable_route_importance_scores.items(), key=lambda item: item[1],reverse=True))

dynamic_route_importance_scores = {a:b for a,b in cluster_importance.items() for a in dynamic_route_clusters}
dynamic_route_importance_scores = dict(sorted(dynamic_route_importance_scores.items(), key=lambda item: item[1],reverse=True))

print("Importance scores calculated")
print("Stable route importance scores:",stable_route_importance_scores)
print("Dynamic route importance scores:",dynamic_route_importance_scores)

In [4]:
'''creating a cluster data for the evaluation of the heuristic solution'''


weekly_demand_file = pd.DataFrame(weekly_cluster_demand.values())

cluster_data = pd.DataFrame({"cluster_id":range(1,total_clusters+1),
                             "priority":cluster_importance.values(),
                             "Week_1_demand_inLM":weekly_demand_file.iloc[:,0],
                             "Week_2_demand_inLM":weekly_demand_file.iloc[:,1],
                             "Week_3_demand_inLM":weekly_demand_file.iloc[:,2],
                             "Week_4_demand_inLM":weekly_demand_file.iloc[:,3],
                             "stable_route_status":stable_route_info.values(),
                             "dynamic_route_status":dynamic_route_info.values(),})
# sorted_cluster_data = cluster_data.sort_values(by=["priority"],ascending=False,ignore_index=True)
truck_capacities = [truck_capacity*i for i in range(1,trucks+1)]
# print("Truck capacities:",truck_capacities)
cluster_data["best_truck_capacity"] = cluster_truck_capacity.values()

In [5]:
'''creating the total leftovers column for the evaluation of the heuristic solution'''''

time = range(1,weeks+1)
for week in time:
    shortage_col = f"W{week}_leftovers"
    week_demand_col = f"Week_{week}_demand_inLM"
    cluster_data[shortage_col] = np.where(cluster_data[week_demand_col] > cluster_data["best_truck_capacity"],
                                          abs(cluster_data["best_truck_capacity"]-cluster_data[week_demand_col]),0)
cluster_data["Total_leftovers"] = cluster_data["W1_leftovers"]+cluster_data["W2_leftovers"]+cluster_data["W3_leftovers"]+cluster_data["W4_leftovers"]


In [6]:
'''creating the stable routes column to know how many stable routes are there in the solution''

mask = cluster_data["stable_route_status"]==1
cluster_data.loc[mask,"stable_routes"] = cluster_data["best_truck_capacity"] / truck_capacity

cluster_data.loc[~mask,"stable_routes"] = 0
cluster_data["stable_routes"] = cluster_data["stable_routes"].astype(int)

cluster_data = cluster_data.sort_values(by=["priority"],ascending=False,ignore_index=True)

In [8]:
'''creating the dynamic routes column to know how many dynamic routes are there in the solution'''

conditions = []
choices_dynamic_routes = []
for capacity in truck_capacities:
    condition = (cluster_data["Total_leftovers"] < capacity) & (cluster_data["Total_leftovers"] >= loading_capacity_dynamic)
    conditions.append(condition)
    choices_dynamic_routes.append(int(capacity/truck_capacity))


conditions.append((cluster_data["Total_leftovers"] < loading_capacity_dynamic) & (cluster_data["Total_leftovers"] > 0))
choices_dynamic_routes.append(0)

conditions.append(cluster_data["dynamic_route_status"]==0)
choices_dynamic_routes.append(0)

conditions.append(cluster_data["dynamic_route_status"]==1)
choices_dynamic_routes.append(cluster_data["best_truck_capacity"] / truck_capacity)

# conditions.append((cluster_data["Total_leftovers"] < loading_capacity_dynamic) & (cluster_data["Total_leftovers"] > 0) & (cluster_data["dynamic_route_status"]==1))
# choices_dynamic_routes.append((cluster_data["best_truck_capacity"] / truck_capacity))

cluster_data["dynamic_routes"] = np.select(conditions,choices_dynamic_routes,default=(cluster_data["best_truck_capacity"] / truck_capacity))
cluster_data["dynamic_routes"] = cluster_data["dynamic_routes"].astype(int)

In [21]:
'''creating the actual shortage column to evaluate the shortage in the solution'''


cluster_data["Shortage"] = np.where(cluster_data["dynamic_routes"] == 0,cluster_data["Total_leftovers"],0)
# pd.DataFrame(cluster_data["Shortage"]).to_csv("Shortage.csv")
cluster_supplier_info = {c+1:set(np.where(matrix.iloc[:,c]==1)[0]+1) for c in range(total_clusters)}
# len(cluster_supplier_info[958])
cluster_data["suppliers_in_cluster"] = cluster_data["cluster_id"].map(cluster_supplier_info)
# cluster_data['suppliers_in_cluster'] = cluster_data['suppliers_in_cluster'].

cluster_data.to_excel("cluster_data.xlsx",index=False)

In [ ]:
'''Some analysis metrics that can be used to evaluate the solution'''
total_stable_routes = cluster_data[cluster_data["stable_route_status"]==1].loc[:,"stable_routes"].sum()
total_dynamic_routes = cluster_data[cluster_data["dynamic_routes"]!=0].loc[:,"dynamic_routes"].sum()
shortage_after_routing_selection = sum([i for i in cluster_data['Shortage']])
print("Total stable routes:",total_stable_routes)
print("Total dynamic routes:",total_dynamic_routes)
print("Total shortage after assigning dynamic (in Lm):",shortage_after_routing_selection)
# sum of shortage values less than loading capacity of dynamic routes (3 LM)

In [12]:
def priority_based_greedy(cluster_supplier_matrix, priority_scores,n_clusters):
    """
    Select clusters based on priority scores while ensuring unique suppliers in the selected clusters.
    
    Parameters:
    - cluster_supplier_matrix: Supplier-cluster relationship matrix
    - priority_scores: Dictionary of cluster priorities
    - n_clusters: Number of clusters
    
    Returns:
    - Dictionary representing selected clusters and their suppliers
    """
    # Create a dictionary of cluster-column ranges
    cluster_column_ranges = {c+1:set(np.where(cluster_supplier_matrix.iloc[:,c]==1)[0]+1) for c in range(n_clusters)}
    # Sort clusters based on priority scores
    sorted_clusters = dict(sorted(priority_scores.items(), key=lambda item: item[1],reverse=True))
    # Sort the already sorted cluster list based on number of suppliers
    sorted_clusters = dict(sorted(sorted_clusters.items(), key=lambda item: (len(cluster_column_ranges[item[0]])),reverse=True))
    # Initialize the solution dictionary
    init_sol = {}
    # Initialize a set of selected suppliers
    selected_suppliers = set()
    # Iterate over the sorted clusters
    for cluster, score in sorted_clusters.items():
        # Check if the cluster is not already in the solution
            if cluster not in init_sol.keys():
                # Check if the cluster has any suppliers that are not already selected
                uncovered_suppliers = cluster_column_ranges[cluster] - selected_suppliers
                # If there are any such suppliers, add them to the solution
                if not any(supplier in selected_suppliers for supplier in cluster_column_ranges[cluster]):
                    # Add the cluster to the solution
                    init_sol[cluster] = list(cluster_column_ranges[cluster])
                    # Add the suppliers to the selected suppliers set
                    selected_suppliers |= cluster_column_ranges[cluster]
    return init_sol
initial_solution = priority_based_greedy(matrix,cluster_importance,total_clusters)

print(initial_solution)
print(len(initial_solution))
selected_clusters  = list(initial_solution.keys())
initial_solution_data = cluster_data[cluster_data["cluster_id"].isin(selected_clusters)].reset_index(drop=True)
# initial_solution_data
    
selected_suppliers = list(initial_solution.values())
selected_suppliers_flatten = [item for sublist in selected_suppliers for item in sublist]

'''this is to check if all the suppliers are covered and none has been repeated'''
print(len(selected_suppliers_flatten))
print((set(selected_suppliers_flatten)))

'''this is to check how many clusters of single supplier are there and how many clusters of multiple suppliers are there, the reason behind this is to
check if the solution is balanced or not and not many single supplier clusters are selected'''

single = []
more = []
for i,k in enumerate(selected_suppliers):
    if len(k)==1:
        single.append(i)
    else:
        more.append(i)
print(len(single))
print(len(more))


In [ ]:
'''Some analysis metrics that can be used to evaluate the solution'''
init_total_stable_routes = len(initial_solution_data[initial_solution_data["stable_route_status"]==1])
# init_total_dynamic_routes = initial_solution_data[initial_solution_data["dynamic_routes"]!=0].loc[:,"stable_route_status"].sum()
init_shortage_after_routing_selection = sum([i for i in initial_solution_data['Shortage']])
suppliers_in_stable_routes = 0
suppliers_in_dynamic_routes = 0
suppliers_in_shortage = 0
for i in range(len(initial_solution_data)):
    if initial_solution_data.loc[i,"stable_routes"] > 0 and initial_solution_data.loc[i, "dynamic_routes"] == 0:
        suppliers_in_stable_routes += len(initial_solution_data.loc[i,"suppliers_in_cluster"])
    elif initial_solution_data.loc[i,"dynamic_routes"] > 0 and initial_solution_data.loc[i,"stable_routes"] == 0:
        suppliers_in_dynamic_routes += len(initial_solution_data.loc[i,"suppliers_in_cluster"])
    elif initial_solution_data.loc[i,"dynamic_routes"] > 0 and initial_solution_data.loc[i,"stable_route_status"] > 0:
        suppliers_in_dynamic_routes += len(initial_solution_data.loc[i,"suppliers_in_cluster"])
        suppliers_in_stable_routes += len(initial_solution_data.loc[i,"suppliers_in_cluster"])

    if initial_solution_data.loc[i,"Shortage"]>0:
        suppliers_in_shortage += len(initial_solution_data.loc[i,"suppliers_in_cluster"])
print("Total suppliers in stable routes:",suppliers_in_stable_routes)
print("Total suppliers in dynamic routes:",suppliers_in_dynamic_routes)
print("Total stable routes of initial solution: ",init_total_stable_routes)
print("Total weekly shortage after assiging dynamic routes of inital solution (in Lm): ",init_shortage_after_routing_selection)
print("Total suppliers in shortage:",suppliers_in_shortage)

In [15]:
'''creating different lists for the result analysis'''

Region = []
suppliers_sr = []
suppliers_dr= []
suppliers_sh = []
init_sr = []
init_sh = []
suppliers = []
percentage_sr_suppliers = []
percentage_dr_suppliers = []  
percentage_sr_sh = []
plant = []
demand = []
supply = []


In [16]:
'''appending the results to the lists created above. Note: the region will be appended manually as it is not a part of the solution'''
Region.append(3)
suppliers_sr.append(suppliers_in_stable_routes)
suppliers_dr.append(suppliers_in_dynamic_routes)
suppliers_sh.append(suppliers_in_shortage)
init_sr.append(init_total_stable_routes)
init_sh.append(init_shortage_after_routing_selection)
suppliers.append(total_suppliers)
percentage_sr_suppliers.append(round((suppliers_in_stable_routes/total_suppliers)*100,2))
percentage_dr_suppliers.append(round((suppliers_in_dynamic_routes/total_suppliers)*100,2))
percentage_sr_sh.append(round((suppliers_in_shortage/total_suppliers)*100,2))


plant.append(plants)
demand.append(sum([i for i in demand_info["need"]]))
supply.append(sum([i for i in supply_info["stock init"]]))

In [ ]:
'''creating a dataframe to store the results'''

results_data = pd.DataFrame({"Region":Region,
                             "Suppliers_in_stable_routes":suppliers_sr,
                             "Suppliers_in_dynamic_routes":suppliers_dr,
                             "Total_suppliers":suppliers,
                             "Percentage_of_suppliers_in_stable_routes":percentage_sr_suppliers,
                             "Percentage_of_suppliers_in_dynamic_routes":percentage_dr_suppliers,
                             "Suppliers_in_shortage":suppliers_sh,
                             "Percentage_of_suppliers_in_shortage":percentage_sr_sh,
                             "Total_stable_routes":init_sr,
                             "Weekly_shortage(inLM)":init_sh},index=[0])
results_data

In [17]:
'''saving the results in an excel file'''
results_data.to_excel("results_data_reg4.xlsx",index=False)

In [ ]:
'''creating a dataframe to store the initial solution data'''
initial_solution_data

In [19]:
'''saving the initial solution data in an excel file'''

initial_solution_data.to_excel("initial_solution_data_reg4.xlsx")

In [ ]:
'''creating a dataframe to store the cluster data'''

data=pd.DataFrame({"Region":Region,
                   "Plants":plant,
                   "Suppliers":suppliers,
                   "TotalDemand":demand,
                   "TotalSupply":supply,},index=[0])
data


In [21]:
'''saving the cluster data in an excel file'''
data.to_excel("data_reg4.xlsx",index=False)